In [6]:
# Import packages and set numpy random seed
import numpy as np
import cv2
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential

#for GPU training
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.33)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1511132824322969920
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3173331764
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18311918145524183777
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 960M, pci bus id: 0000:02:00.0, compute capability: 5.0"
]


In [13]:


model = Sequential(
[
    keras.Input((128,128,3)),
    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(16, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(8, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(4, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(1, activation='sigmoid')
]
)

optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True) #from_logits=True means output probabilities are not normalized
acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()

model.summary()
model.load_weights('data_aug_off_model.h5')


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 128, 32)      2432      
_________________________________________________________________
batch_normalization_5 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 32)        25632     
_________________________________________________________________
batch_normalization_6 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 32)       

In [14]:
@tf.function
def test_step(x):
    val_preds = model(x, training=False)    
    return val_preds

In [20]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray=[gray,gray,gray]
    to_test = np.resize(gray,(1,128,128,3))
    print(to_test)
#     print(to_test.shape)
    result = test_step(to_test)
    result = np.reshape(tf.get_static_value(result),(1,1))[0][0]
    
    # Display the resulting frame
    prediction = ''
    if result>0.5:
        prediction = 'non-drowsy'
    else:
        prediction = 'drowsy'
       
    cv2.putText(frame, str(result), (10, 160), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3) 
    cv2.putText(frame, prediction, (10, 200), cv2.FONT_HERSHEY_PLAIN, 2, (255,255,255), 3) 
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

[[[[150 148 139]
   [146 148 145]
   [145 145 145]
   ...
   [177 180 181]
   [180 170 174]
   [179 184 176]]

  [[175 180 180]
   [178 172 176]
   [180 180 182]
   ...
   [164 166 168]
   [166 164 162]
   [156 163 162]]

  [[162 163 162]
   [159 157 159]
   [159 159 162]
   ...
   [181 171 171]
   [174 173 174]
   [178 178 176]]

  ...

  [[151 149 153]
   [150 142 144]
   [134 130 142]
   ...
   [193 186 186]
   [182 184 191]
   [188 188 187]]

  [[185 187 187]
   [187 189 187]
   [187 187 189]
   ...
   [164 161 161]
   [161 163 166]
   [168 168 167]]

  [[170 177 178]
   [178 178 178]
   [178 178 180]
   ...
   [ 67  68  68]
   [ 83  93  92]
   [169 252 252]]]]
[[[[149 147 145]
   [140 145 144]
   [145 128 146]
   ...
   [176 177 180]
   [180 181 179]
   [178 177 178]]

  [[181 181 181]
   [179 179 179]
   [175 176 177]
   ...
   [177 172 171]
   [169 165 163]
   [156 160 162]]

  [[162 159 158]
   [158 161 162]
   [164 164 163]
   ...
   [170 169 171]
   [171 171 171]
   [173 175 

   [173 252 252]]]]
[[[[139 141 142]
   [144 145 144]
   [145 150 154]
   ...
   [178 179 172]
   [171 185 178]
   [178 179 179]]

  [[179 178 176]
   [179 178 176]
   [176 177 177]
   ...
   [167 163 169]
   [165 160 157]
   [162 163 161]]

  [[159 157 158]
   [159 156 155]
   [155 160 153]
   ...
   [183 178 170]
   [169 173 171]
   [170 167 170]]

  ...

  [[135 137 144]
   [141 150 144]
   [137 138 144]
   ...
   [180 186 194]
   [191 191 191]
   [189 186 186]]

  [[186 186 188]
   [186 186 186]
   [184 185 186]
   ...
   [165 166 165]
   [167 168 170]
   [180 172 164]]

  [[168 172 183]
   [191 185 181]
   [180 182 171]
   ...
   [ 70  67  65]
   [ 84  84  89]
   [173 252 252]]]]
[[[[145 154 137]
   [134 149 135]
   [142 144 146]
   ...
   [176 177 178]
   [178 178 174]
   [183 186 179]]

  [[179 180 179]
   [175 174 183]
   [175 174 178]
   ...
   [163 175 166]
   [162 162 160]
   [162 164 162]]

  [[154 157 155]
   [154 161 160]
   [160 161 162]
   ...
   [171 171 171]
   [170 1

[[[[141 140 144]
   [144 140 148]
   [129 142 143]
   ...
   [181 185 175]
   [175 172 173]
   [179 178 179]]

  [[185 184 178]
   [175 177 180]
   [176 173 174]
   ...
   [171 175 170]
   [171 165 166]
   [173 154 159]]

  [[160 159 161]
   [162 162 162]
   [165 168 165]
   ...
   [175 173 171]
   [171 174 179]
   [172 171 168]]

  ...

  [[138 137 148]
   [143 144 144]
   [151 148 144]
   ...
   [187 181 179]
   [185 190 187]
   [186 187 187]]

  [[187 187 186]
   [186 184 187]
   [180 180 183]
   ...
   [164 164 167]
   [167 167 166]
   [166 168 169]]

  [[170 177 178]
   [179 180 182]
   [178 178 176]
   ...
   [ 72  74  69]
   [ 84  86  92]
   [164 253 253]]]]
[[[[141 140 144]
   [144 140 148]
   [129 142 143]
   ...
   [181 185 175]
   [175 172 173]
   [179 178 179]]

  [[185 184 178]
   [175 177 180]
   [176 173 174]
   ...
   [171 175 170]
   [171 165 166]
   [173 154 159]]

  [[160 159 161]
   [162 162 162]
   [165 168 165]
   ...
   [175 173 171]
   [171 174 179]
   [172 171 

[[[[147 148 153]
   [146 141 144]
   [148 152 148]
   ...
   [178 177 169]
   [176 181 179]
   [175 176 174]]

  [[172 180 166]
   [174 179 180]
   [178 176 178]
   ...
   [170 177 170]
   [171 162 160]
   [160 159 158]]

  [[158 160 161]
   [160 162 162]
   [155 159 161]
   ...
   [173 175 172]
   [171 172 171]
   [174 179 169]]

  ...

  [[144 146 147]
   [150 149 145]
   [139 147 154]
   ...
   [187 186 186]
   [187 187 187]
   [187 187 183]]

  [[184 185 181]
   [183 187 189]
   [188 187 182]
   ...
   [165 166 163]
   [164 164 164]
   [165 169 171]]

  [[169 169 173]
   [177 181 179]
   [175 174 176]
   ...
   [ 73  77  75]
   [ 80  91  93]
   [175 251 251]]]]
[[[[147 148 153]
   [146 141 144]
   [148 152 148]
   ...
   [178 177 169]
   [176 181 179]
   [175 176 174]]

  [[172 180 166]
   [174 179 180]
   [178 176 178]
   ...
   [170 177 170]
   [171 162 160]
   [160 159 158]]

  [[158 160 161]
   [160 162 162]
   [155 159 161]
   ...
   [173 175 172]
   [171 172 171]
   [174 179 

[[[[142 133 145]
   [152 149 145]
   [145 145 143]
   ...
   [179 182 180]
   [182 182 174]
   [174 176 175]]

  [[176 180 179]
   [177 178 176]
   [176 175 172]
   ...
   [169 167 166]
   [170 156 155]
   [154 156 157]]

  [[154 157 162]
   [159 155 160]
   [161 160 160]
   ...
   [171 170 171]
   [170 175 176]
   [171 173 171]]

  ...

  [[147 154 154]
   [150 144 161]
   [154 148 149]
   ...
   [184 184 185]
   [186 186 191]
   [184 182 186]]

  [[187 187 187]
   [187 189 190]
   [185 189 190]
   ...
   [165 164 170]
   [174 170 169]
   [168 169 171]]

  [[166 168 179]
   [180 178 182]
   [179 176 177]
   ...
   [ 69  72  75]
   [ 93  95  94]
   [138 253 253]]]]
[[[[138 141 146]
   [145 146 147]
   [148 150 143]
   ...
   [176 176 174]
   [176 178 179]
   [177 174 168]]

  [[173 178 175]
   [171 173 174]
   [176 178 180]
   ...
   [169 164 166]
   [167 167 159]
   [157 156 153]]

  [[163 163 159]
   [155 154 158]
   [163 168 160]
   ...
   [172 172 178]
   [173 168 169]
   [168 170 

...

[[[[137 140 139]
   [142 150 146]
   [141 147 139]
   ...
   [170 180 179]
   [173 173 171]
   [172 174 170]]

  [[170 171 174]
   [171 169 176]
   [170 169 175]
   ...
   [170 177 168]
   [165 161 157]
   [155 154 156]]

  [[152 160 159]
   [159 159 159]
   [159 158 159]
   ...
   [172 169 169]
   [171 171 168]
   [167 171 167]]

  ...

  [[145 139 138]
   [145 147 141]
   [140 145 144]
   ...
   [182 183 183]
   [183 180 177]
   [180 183 183]]

  [[182 185 187]
   [188 189 185]
   [186 185 184]
   ...
   [162 162 161]
   [162 164 168]
   [169 170 169]]

  [[161 172 181]
   [177 176 177]
   [177 177 178]
   ...
   [ 65  65  63]
   [ 74  89  96]
   [169 251 251]]]]
[[[[154 140 139]
   [141 133 145]
   [149 141 138]
   ...
   [174 176 170]
   [170 174 176]
   [179 174 171]]

  [[176 177 175]
   [173 169 173]
   [175 172 173]
   ...
   [168 170 170]
   [163 157 155]
   [157 150 156]]

  [[155 162 162]
   [157 156 158]
   [157 149 153]
   ...
   [169 168 167]
   [170 170 171]
   [169 166 

[[[[145 146 146]
   [139 148 149]
   [145 141 142]
   ...
   [176 172 172]
   [177 176 174]
   [174 173 172]]

  [[171 174 173]
   [173 174 174]
   [174 173 174]
   ...
   [167 166 161]
   [160 154 155]
   [159 158 158]]

  [[159 159 158]
   [158 153 152]
   [152 158 155]
   ...
   [170 169 169]
   [169 171 174]
   [173 169 170]]

  ...

  [[153 149 146]
   [151 150 148]
   [145 143 147]
   ...
   [179 183 187]
   [181 181 183]
   [187 189 190]]

  [[184 184 184]
   [184 184 184]
   [183 184 184]
   ...
   [163 163 162]
   [163 164 163]
   [165 165 168]]

  [[167 170 172]
   [177 177 177]
   [175 178 177]
   ...
   [ 66  62  70]
   [ 70  75  82]
   [137 252 254]]]]
[[[[145 145 144]
   [144 148 146]
   [135 138 136]
   ...
   [177 180 171]
   [174 179 176]
   [179 174 172]]

  [[174 173 174]
   [170 166 172]
   [175 170 171]
   ...
   [169 167 162]
   [163 164 157]
   [152 152 153]]

  [[157 158 158]
   [156 157 158]
   [160 158 161]
   ...
   [167 186 172]
   [165 170 173]
   [172 170 

[[[[144 143 142]
   [144 147 153]
   [150 149 137]
   ...
   [172 168 170]
   [174 174 167]
   [174 170 169]]

  [[182 184 175]
   [176 177 172]
   [171 174 176]
   ...
   [169 171 166]
   [165 165 163]
   [160 158 157]]

  [[157 162 162]
   [160 155 158]
   [164 162 159]
   ...
   [173 177 171]
   [169 166 165]
   [170 172 170]]

  ...

  [[145 146 147]
   [151 149 147]
   [144 142 142]
   ...
   [185 180 183]
   [184 182 182]
   [185 185 184]]

  [[183 183 187]
   [179 186 187]
   [186 186 187]
   ...
   [162 164 165]
   [166 165 165]
   [167 167 169]]

  [[170 172 178]
   [178 179 183]
   [181 177 173]
   ...
   [ 65  72  75]
   [ 81  80  90]
   [163 252 252]]]]
[[[[144 143 142]
   [144 147 153]
   [150 149 137]
   ...
   [172 168 170]
   [174 174 167]
   [174 170 169]]

  [[182 184 175]
   [176 177 172]
   [171 174 176]
   ...
   [169 171 166]
   [165 165 163]
   [160 158 157]]

  [[157 162 162]
   [160 155 158]
   [164 162 159]
   ...
   [173 177 171]
   [169 166 165]
   [170 172 

[[[[138 142 145]
   [143 142 145]
   [146 145 142]
   ...
   [177 176 174]
   [174 176 174]
   [174 174 174]]

  [[172 170 172]
   [175 178 177]
   [177 175 175]
   ...
   [168 162 164]
   [167 163 153]
   [158 161 162]]

  [[162 160 158]
   [159 162 158]
   [158 162 157]
   ...
   [171 166 164]
   [171 171 167]
   [169 168 168]]

  ...

  [[149 145 146]
   [148 147 144]
   [144 147 146]
   ...
   [190 181 177]
   [180 177 178]
   [184 182 182]]

  [[182 184 185]
   [185 185 183]
   [183 183 184]
   ...
   [165 164 162]
   [165 173 178]
   [171 169 168]]

  [[165 162 173]
   [168 170 178]
   [177 172 170]
   ...
   [ 69  69  65]
   [ 72  79  89]
   [130 251 252]]]]
[[[[132 140 140]
   [139 134 146]
   [145 135 138]
   ...
   [175 174 177]
   [175 173 174]
   [177 174 172]]

  [[170 169 174]
   [177 178 178]
   [182 170 176]
   ...
   [167 169 170]
   [161 156 156]
   [155 153 153]]

  [[161 154 154]
   [155 158 159]
   [160 160 158]
   ...
   [171 175 172]
   [171 171 171]
   [171 170 

[[[[148 146 137]
   [144 151 141]
   [140 137 137]
   ...
   [170 173 177]
   [175 175 176]
   [175 176 176]]

  [[170 174 177]
   [177 177 180]
   [176 173 176]
   ...
   [167 175 175]
   [169 161 153]
   [154 156 155]]

  [[153 156 160]
   [160 160 161]
   [158 152 159]
   ...
   [173 170 170]
   [170 169 169]
   [173 177 169]]

  ...

  [[139 140 143]
   [141 145 142]
   [137 146 147]
   ...
   [185 187 185]
   [182 181 181]
   [182 184 184]]

  [[184 186 188]
   [186 186 186]
   [187 187 185]
   ...
   [158 160 160]
   [163 161 164]
   [175 170 165]]

  [[165 170 174]
   [177 176 175]
   [177 177 169]
   ...
   [ 65  70  72]
   [ 75  81  85]
   [174 252 252]]]]
[[[[148 146 137]
   [144 151 141]
   [140 137 137]
   ...
   [170 173 177]
   [175 175 176]
   [175 176 176]]

  [[170 174 177]
   [177 177 180]
   [176 173 176]
   ...
   [167 175 175]
   [169 161 153]
   [154 156 155]]

  [[153 156 160]
   [160 160 161]
   [158 152 159]
   ...
   [173 170 170]
   [170 169 169]
   [173 177 